In [38]:
from tavily import TavilyClient
from scrape_metadata import scrape_metadata
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
import json
import re

In [21]:
query = 'horror movies made in morocco'
client = TavilyClient("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")
response = client.search(
    query=query,
    search_depth='advanced',
    max_results=5,
)

In [ ]:
template = """
    You will get two pieces of text:

    1. A search query:
    {query}

    2. Web‑crawled text results:
    {results}

    Your task is to:
    - read the results
    - recognize all the mentioned movies
    - within these movies, understand which ones are actually an answer to the query
    - return a JSON object with the following structure:
    {{"movies": [{{
        "title": "Movie Title 1",
        "year": "YYYY" (optional)
        }},
        {{
        "title": "Movie Title 2",
        "year": "YYYY" (optional)
        }}
    ]
    }}

    IMPORTANT: You should not include any other information in the response, just the JSON object in a way that i can parse it immediately.
    Don't introduce your answer or write "json" in the beginning. Just start with the first curly brace of the JSON object.
    """
prompt = PromptTemplate.from_template(template=template)
llm = ChatOllama(model="gemma3:4b", temperature=0)

chain = prompt | llm

In [33]:
ai_res = chain.invoke(input = {"query": query, "results": response['results']})

In [39]:
json_str = re.search(r'\{.*\}', ai_res.content).group(0)
json_obj = json.loads(json_str, strict=False)
print(json_obj)

{'movies': [{'title': 'Achoura', 'year': '2018'}, {'title': 'La Damnée', 'year': '2024'}, {'title': 'Kandisha', 'year': '2008'}, {'title': 'Mirages', 'year': '2010'}, {'title': 'The Pyramid', 'year': '2014'}]}
